1. Delete the columns Time, Sub_metering_2 and Sub_metering_3

In [3]:
import pandas as pd

data = pd.read_csv(
    "../data/household_power_consumption.txt", 
    sep=";", 
    low_memory =False
)

data_drop = data.drop(['Time', 'Sub_metering_2', 'Sub_metering_3'], axis=1)

print(data_drop)

             Date Global_active_power Global_reactive_power  Voltage  \
0      16/12/2006               4.216                 0.418  234.840   
1      16/12/2006               5.360                 0.436  233.630   
2      16/12/2006               5.374                 0.498  233.290   
3      16/12/2006               5.388                 0.502  233.740   
4      16/12/2006               3.666                 0.528  235.680   
...           ...                 ...                   ...      ...   
65639   31/1/2007               0.698                 0.134  240.810   
65640   31/1/2007               0.706                 0.134  240.600   
65641   31/1/2007               0.698                 0.122  241.600   
65642   31/1/2007               0.638                 0.000  241.460   
65643   31/1/2007                 NaN                   NaN      NaN   

      Global_intensity Sub_metering_1  
0               18.400          0.000  
1               23.000          0.000  
2              

2. Set Date as index

In [4]:
df = pd.DataFrame(data_drop)
df_indexed = df.set_index('Date')

print(df_indexed)

           Global_active_power Global_reactive_power  Voltage  \
Date                                                            
16/12/2006               4.216                 0.418  234.840   
16/12/2006               5.360                 0.436  233.630   
16/12/2006               5.374                 0.498  233.290   
16/12/2006               5.388                 0.502  233.740   
16/12/2006               3.666                 0.528  235.680   
...                        ...                   ...      ...   
31/1/2007                0.698                 0.134  240.810   
31/1/2007                0.706                 0.134  240.600   
31/1/2007                0.698                 0.122  241.600   
31/1/2007                0.638                 0.000  241.460   
31/1/2007                  NaN                   NaN      NaN   

           Global_intensity Sub_metering_1  
Date                                        
16/12/2006           18.400          0.000  
16/12/2006         

Create a function that takes as input the DataFrame with the data set and returns a DataFrame with updated types:

```
    def update_types(df):
        #TODO
        return df

In [87]:
import pandas as pd

def update_types(d):
    """
    Automatically converts DataFrame columns to the most appropriate types:
    - Converts numeric strings to floats/ints
    - Keeps list columns as object
    - Keeps text columns as string
    """
    for col in d.columns:
        # Try to convert to numeric (if possible)
        try:
            d[col] = pd.to_numeric(d[col])
        except (ValueError, TypeError):
            # Keep as-is if not numeric
            pass
    return d

Use describe to have an overview on the data set

In [21]:
df_indexed.describe(percentiles=None, include=None, exclude=None)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1
count,65643,65643,65643,65643,65643
unique,3238,367,2148,190,61
top,0.216,0.000,242.100,1.400,0.000
freq,487,17698,109,4678,60427


5. Delete the rows with missing values

In [105]:
df_clean = update_types(df_indexed.dropna())

print(df_clean)

           Global_active_power Global_reactive_power  Voltage  \
Date                                                            
16/12/2006               4.216                 0.418  234.840   
16/12/2006               5.360                 0.436  233.630   
16/12/2006               5.374                 0.498  233.290   
16/12/2006               5.388                 0.502  233.740   
16/12/2006               3.666                 0.528  235.680   
...                        ...                   ...      ...   
31/1/2007                0.700                 0.136  240.840   
31/1/2007                0.698                 0.134  240.810   
31/1/2007                0.706                 0.134  240.600   
31/1/2007                0.698                 0.122  241.600   
31/1/2007                0.638                 0.000  241.460   

           Global_intensity Sub_metering_1  
Date                                        
16/12/2006           18.400          0.000  
16/12/2006         

6. Modify Sub_metering_1 by adding 1 to it and multiplying the total by 0.06. If x is a row the output is: (x+1)*0.06

In [119]:
type(df_updated['Sub_metering_1'].iloc[0])

df_updated['Sub_metering_1'] = (df_updated['Sub_metering_1'] + 1) * 0.06

print(df_updated)


           Global_active_power Global_reactive_power  Voltage  \
Date                                                            
16/12/2006               4.216                 0.418  234.840   
16/12/2006               5.360                 0.436  233.630   
16/12/2006               5.374                 0.498  233.290   
16/12/2006               5.388                 0.502  233.740   
16/12/2006               3.666                 0.528  235.680   
...                        ...                   ...      ...   
31/1/2007                0.700                 0.136  240.840   
31/1/2007                0.698                 0.134  240.810   
31/1/2007                0.706                 0.134  240.600   
31/1/2007                0.698                 0.122  241.600   
31/1/2007                0.638                 0.000  241.460   

           Global_intensity  Sub_metering_1  
Date                                         
16/12/2006           18.400          0.1236  
16/12/2006      

/tmp/ipykernel_16461/3264405612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_updated['Sub_metering_1'] = (df_updated['Sub_metering_1'] + 1) * 0.06
